In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from pathlib import Path

In [4]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi
from exact_sync.v1.api.teams_api import TeamsApi

from exact_sync.v1.models import ImageSet, Team, Product, AnnotationType, Image, Annotation, AnnotationMediaFile
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

In [5]:
configuration = Configuration()
configuration.username = 'marzahl'
configuration.password = 'imagetagger'
configuration.host =  "https://exact.cs.fau.de/"

image_set_ids = [15, 245]

In [6]:
client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

In [7]:
images = {}
annotation_types = {}

for image_set in image_set_ids:
    for image in images_api.list_images(pagination=False, image_set=image_set).results:
        images[image.id] = image
    for product in product_api.list_products(imagesets=image_set).results:
        for anno_type in annotation_types_api.list_annotation_types(product=product.id).results:
            annotation_types[anno_type.id] = anno_type

In [8]:
rows = []
for image_id, image in tqdm(images.items()):

    for annotation_type_id, annotation_type in annotation_types.items():

        for anno in annotations_api.list_annotations(annotation_type=annotation_type_id, image=image_id, pagination=False, fields="id,unique_identifier,annotation_type,vector,user,deleted,last_editor").results:
            
            species = "equine"
            data_set_name = "SREP"
            version = "SREP"
            
            # anno_id, image_id, image_name, anno_type_name, vector, user_id, user_name
            image_type = "Prussian" if "erliner" in image.name else "Turnbull"
            if anno.vector is not None and anno.user in [1, 11, 12]:
                rows.append([anno.id, image_id, image.image_set, species, image.name, image_type, int(annotation_type.name), anno.vector, anno.unique_identifier, anno.user, anno.deleted, anno.last_editor, data_set_name, version])

    df = pd.DataFrame(rows, columns=['id', 'image_id', 'image_set', 'species' ,'image_name', 'image_type', 'grade', 'vector', 'unique_identifier', 'user_id', 'deleted', 'last_editor', 'data_set_name', 'version'])
    df.to_pickle("EIPH_Equine_SREP_Annotations.pkl")

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [06:26<00:00, 22.72s/it]


In [9]:
rows = []
for image_id, image in tqdm(images.items()):

    for annotation_type_id, annotation_type in annotation_types.items():

        for anno in annotations_api.list_annotations(annotation_type=annotation_type_id, 
                                                     image=image_id, pagination=False, deleted=False,
                                                     fields="id,unique_identifier,annotation_type,vector,user,deleted,last_editor").results:
            
            species = "equine"
            data_set_name = "SREP"
            version = "Screened"
            
            # anno_id, image_id, image_name, anno_type_name, vector, user_id, user_name
            image_type = "Prussian" if "erliner" in image.name else "Turnbull"
            if anno.vector is not None and anno.user in [1, 11, 12]:
                rows.append([anno.id, image_id, image.image_set, species, image.name, image_type, int(annotation_type.name), anno.vector, anno.unique_identifier, anno.user, anno.deleted, anno.last_editor, data_set_name, version])

    df = pd.DataFrame(rows, columns=['id', 'image_id', 'image_set', 'species' ,'image_name', 'image_type', 'grade', 'vector', 'unique_identifier', 'user_id', 'deleted', 'last_editor', 'data_set_name', 'version'])
    df.to_pickle("EIPH_Equine_SREP_Screening_Annotations.pkl")

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [05:00<00:00, 17.68s/it]


In [10]:
rows = []
for image_set in [269, 240]:
    for image in tqdm(images_api.list_images(pagination=False, image_set=image_set).results):

        for annotation_type_id, annotation_type in annotation_types.items():

            for anno in annotations_api.list_annotations(annotation_type=annotation_type_id, 
                                                         image=image.id, pagination=False, deleted=False,
                                                         fields="id,unique_identifier,annotation_type,vector,user,deleted,last_editor").results:

                species = "equine"
                data_set_name = "SREP"
                version = "Density"

                # anno_id, image_id, image_name, anno_type_name, vector, user_id, user_name
                image_type = "Prussian" if "erliner" in image.name else "Turnbull"
                if anno.vector is not None and anno.user in [1, 11, 12]:
                    rows.append([anno.id, image.id, image.image_set, species, image.name, image_type, int(annotation_type.name), anno.vector, anno.unique_identifier, anno.user, anno.deleted, anno.last_editor, data_set_name, version])

        df = pd.DataFrame(rows, columns=['id', 'image_id', 'image_set', 'species' ,'image_name', 'image_type', 'grade', 'vector', 'unique_identifier', 'user_id', 'deleted', 'last_editor', 'data_set_name', 'version'])
        df.to_pickle("EIPH_Equine_SREP_Density_Annotations.pkl")

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:45<00:00, 22.59s/it]


In [11]:
df

,id,image_id,image_set,species,image_name,image_type,grade,vector,unique_identifier,user_id,deleted,last_editor,data_set_name,version
0,2327744,5204,269,equine,07_EIPH_574882 left lung Turnbull blue.tiff,Turnbull,0,"{'x1': 5, 'x2': 55, 'y1': 15005, 'y2': 15055}",52176097-849f-4f32-ba94-5893e5a86475,1,False,1,SREP,Density
1,2327746,5204,269,equine,07_EIPH_574882 left lung Turnbull blue.tiff,Turnbull,0,"{'x1': 5, 'x2': 55, 'y1': 14945, 'y2': 14995}",6aaee957-79af-4510-9e54-d011c4f1be45,1,False,1,SREP,Density
2,2327748,5204,269,equine,07_EIPH_574882 left lung Turnbull blue.tiff,Turnbull,0,"{'x1': 5, 'x2': 55, 'y1': 14885, 'y2': 14935}",85a3808d-88a7-4056-a7ab-e9f804cdb291,1,False,1,SREP,Density
3,2327750,5204,269,equine,07_EIPH_574882 left lung Turnbull blue.tiff,Turnbull,0,"{'x1': 5, 'x2': 55, 'y1': 14825, 'y2': 14875}",02ec16d8-475f-4504-b8a8-fea5a3535449,1,False,1,SREP,Density
4,2327752,5204,269,equine,07_EIPH_574882 left lung Turnbull blue.tiff,Turnbull,0,"{'x1': 5, 'x2': 55, 'y1': 14765, 'y2': 14815}",15a57471-695e-41fa-b868-95b2bf247dd3,1,False,1,SREP,Density
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59313,2460275,5166,240,equine,20_EIPH_576150 berliner blau.tiff,Prussian,4,"{'x1': 17944, 'x2': 17994, 'y1': 15006, 'y2': ...",c5957b32-fd8b-4c06-99ec-501f5efd5dc1,1,False,12,SREP,Density
59314,2460357,5166,240,equine,20_EIPH_576150 berliner blau.tiff,Prussian,4,"{'x1': 19025, 'x2': 19075, 'y1': 15005, 'y2': ...",4f497654-8d30-483d-9194-b51530ff6dd7,1,False,12,SREP,Density
59315,2460365,5166,240,equine,20_EIPH_576150 berliner blau.tiff,Prussian,4,"{'x1': 19085, 'x2': 19135, 'y1': 14945, 'y2': ...",bbb42a0b-0620-4628-8c2f-df9a8fa08a48,1,False,12,SREP,Density
59316,2460368,5166,240,equine,20_EIPH_576150 berliner blau.tiff,Prussian,4,"{'x1': 19145, 'x2': 19195, 'y1': 15005, 'y2': ...",922f8bcf-9526-44cb-8987-f4369060c16e,1,False,1,SREP,Density
